# Clean Baseline Curve Fitting for Air Filter Restriction

## Objective
Fit a clean baseline curve **R_clean(HP)** for each asset using:
- **Isotonic Regression** for monotonic relationship
- **Linear extrapolation** to cover full operating range
- **5th percentile binning** to capture lower envelope (clean filter behavior)

## Key Challenge
We need to predict air filter restriction across the **entire HP range**, but clean baseline data only exists in lower restriction ranges. Solution: forced extrapolation to asset limits.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.isotonic import IsotonicRegression
import joblib
import matplotlib.pyplot as plt
import os
import csv


# File paths
DATA_PATH = "../data/air_filter_data.csv"
LIMITS_PATH = "../data/asset_limits.csv"

# Processing parameters
CHUNKSIZE = 100_000      # For memory-efficient loading
HP_BIN_WIDTH = 50        # HP bin width for aggregation
PERCENTILE = 5           # Lower envelope (5th percentile)

## Step 1: Load Asset Operating Limits
Each asset has maximum HP and restriction values from engineering specifications.

In [ ]:
# Load asset limits
limits_df = pd.read_csv(LIMITS_PATH)

def normalize_asset_id(val):
    """Handles both float and string asset IDs"""
    try:
        return str(int(float(val)))
    except Exception:
        return str(val)

asset_limits = {
    normalize_asset_id(row['asset']): {
        'max_restriction': row['Max_AirFilterRestriction'],
        'max_hp': row['Max_Horsepower']
    }
    for _, row in limits_df.iterrows()
}

print(f"Loaded limits for {len(asset_limits)} assets")
print(f"Example: Asset limits = {list(asset_limits.items())[:2]}")

## Step 2: Find Assets in Both Datasets
Only process assets present in both air_filter_data.csv and asset_limits.csv

In [ ]:
print("Scanning available assets...")
asset_ids = set()
all_data_asset_ids = set()

for chunk in pd.read_csv(DATA_PATH, usecols=["asset"], chunksize=CHUNKSIZE):
    chunk_ids = set(normalize_asset_id(a) for a in chunk["asset"].unique())
    all_data_asset_ids.update(chunk_ids)
    asset_ids.update(chunk_ids)

print(f"Assets in air_filter_data.csv: {sorted(all_data_asset_ids)}")
print(f"Assets in asset_limits.csv: {sorted(asset_limits.keys())}")

asset_ids = sorted([aid for aid in asset_ids if aid in asset_limits])
print(f"\n✓ Processing {len(asset_ids)} assets: {asset_ids}")

## Step 3: Process Each Asset

### 3.1 Load Asset Data
Load HP and Restriction data for one asset (memory-efficient chunked reading)

In [ ]:
# Select one asset for demonstration
asset_id = asset_ids[0]  # Change index to view different assets
print(f"Processing asset {asset_id}")

# Create output directory
asset_dir = os.path.join("api", asset_id)
os.makedirs(asset_dir, exist_ok=True)

# Load data for this asset
hp_list = []
restriction_list = []

for chunk in pd.read_csv(DATA_PATH, 
                         usecols=["asset", "HydraulicHorsepower", "AirFilterRestriction"], 
                         chunksize=CHUNKSIZE):
    mask = chunk["asset"].astype(str) == asset_id
    if mask.any():
        hp_list.append(chunk.loc[mask, "HydraulicHorsepower"].values)
        restriction_list.append(chunk.loc[mask, "AirFilterRestriction"].values)

hp = np.concatenate(hp_list)
restriction = np.concatenate(restriction_list)

max_hp_asset = asset_limits[asset_id]['max_hp']
max_restriction_asset = asset_limits[asset_id]['max_restriction']

print(f"Loaded {len(hp):,} data points")
print(f"HP range: {np.min(hp):.2f} to {np.max(hp):.2f}")
print(f"Restriction range: {np.min(restriction):.2f} to {np.max(restriction):.2f}")
print(f"Asset limits: HP={max_hp_asset}, Restriction={max_restriction_asset}")

### 3.2 Bin Data and Compute Lower Envelope
- Bin HP into fixed-width intervals
- Compute 5th percentile restriction per bin (clean filter behavior)

In [ ]:
# Create bins
hp_min, hp_max = np.min(hp), np.max(hp)
bins = np.arange(hp_min, max(hp_max, max_hp_asset) + HP_BIN_WIDTH, HP_BIN_WIDTH)
bin_indices = np.digitize(hp, bins) - 1

# Compute 5th percentile per bin
bin_centers = []
bin_restriction = []

for i in range(len(bins) - 1):
    mask = bin_indices == i
    bin_center = (bins[i] + bins[i+1]) / 2
    if np.any(mask):
        perc = np.percentile(restriction[mask], PERCENTILE)
        bin_centers.append(bin_center)
        bin_restriction.append(perc)
    else:
        bin_centers.append(bin_center)
        bin_restriction.append(np.nan)

bin_centers = np.array(bin_centers)
bin_restriction = np.array(bin_restriction)

# Remove empty bins
fit_mask = ~np.isnan(bin_restriction)
fit_bin_centers = bin_centers[fit_mask]
fit_bin_restriction = bin_restriction[fit_mask]

print(f"Created {len(fit_bin_centers)} bins with data")
print(f"HP bin range: {fit_bin_centers.min():.2f} to {fit_bin_centers.max():.2f}")
print(f"Restriction range (5th %ile): {fit_bin_restriction.min():.2f} to {fit_bin_restriction.max():.2f}")

### 3.3 Fit Isotonic Regression
Ensures monotonic relationship: restriction never decreases as HP increases

In [ ]:
print("Fitting Isotonic Regression...")
iso = IsotonicRegression(increasing=True, out_of_bounds="clip")
iso.fit(fit_bin_centers, fit_bin_restriction)

# Get predictions
iso_pred = iso.predict(fit_bin_centers)

print(f"Model fitted:")
print(f"  Input HP range: {fit_bin_centers.min():.2f} to {fit_bin_centers.max():.2f}")
print(f"  Output Restriction range: {iso_pred.min():.2f} to {iso_pred.max():.2f}")

### 3.4 Compute Initial Linear Extrapolation
Use the last rising segment to extrapolate beyond fitted data

In [ ]:
# Find the last segment with significant positive slope
slopes = np.diff(iso_pred) / np.diff(fit_bin_centers)
SLOPE_THRESHOLD = 0.001
still_rising = slopes > SLOPE_THRESHOLD

if np.any(still_rising):
    N_POINTS_FOR_EXTRAP = 10
    last_rising_indices = np.where(still_rising)[0]
    if len(last_rising_indices) >= N_POINTS_FOR_EXTRAP:
        extrap_indices = last_rising_indices[-N_POINTS_FOR_EXTRAP:]
    else:
        extrap_indices = last_rising_indices
    X_extrap = fit_bin_centers[extrap_indices]
    y_extrap = iso_pred[extrap_indices]
    extrapolation_slope, extrapolation_intercept = np.polyfit(X_extrap, y_extrap, 1)
else:
    print("Warning: No rising trend, using last two points")
    X_extrap = fit_bin_centers[-2:]
    y_extrap = iso_pred[-2:]
    extrapolation_slope, extrapolation_intercept = np.polyfit(X_extrap, y_extrap, 1)

print(f"Initial extrapolation from last {len(X_extrap)} points")
print(f"  Slope: {extrapolation_slope:.6f} Restriction per HP")
print(f"  Intercept: {extrapolation_intercept:.6f}")

### 3.5 Force Extrapolation to Asset Limits
**Critical Step**: Adjust slope to ensure we reach max_restriction at max_hp

In [ ]:
hp_at_extrap_start = fit_bin_centers.max()
r_at_extrap_start = iso.predict([[hp_at_extrap_start]])[0]
restriction_at_max_hp_natural = extrapolation_slope * max_hp_asset + extrapolation_intercept

print(f"Natural extrapolation reaches R={restriction_at_max_hp_natural:.2f} at HP={max_hp_asset}")
print(f"Asset limit requires: R={max_restriction_asset} at HP={max_hp_asset}")

if restriction_at_max_hp_natural < max_restriction_asset:
    print(f"\n⚠️ FORCING STEEPER SLOPE to reach asset limits")
    extrapolation_slope = (max_restriction_asset - r_at_extrap_start) / (max_hp_asset - hp_at_extrap_start)
    extrapolation_intercept = r_at_extrap_start - extrapolation_slope * hp_at_extrap_start
    print(f"  Adjusted slope: {extrapolation_slope:.6f}")
    print(f"  Now reaches R={max_restriction_asset} at HP={max_hp_asset}")
    hp_at_max_restriction = max_hp_asset
else:
    print("✓ Natural extrapolation is sufficient")
    if extrapolation_slope > 0:
        hp_at_max_restriction = (max_restriction_asset - extrapolation_intercept) / extrapolation_slope
    else:
        hp_at_max_restriction = np.inf

### 3.6 Define Prediction Function
Combines isotonic model with linear extrapolation

In [ ]:
def predict_restriction_from_hp(hp_values, iso_model, max_fitted_hp, 
                                slope, intercept, max_restriction_limit):
    """Predict restriction from HP using hybrid isotonic + linear model"""
    hp_values = np.atleast_1d(hp_values)
    predictions = np.zeros_like(hp_values, dtype=float)
    
    for i, hp_val in enumerate(hp_values):
        if hp_val <= max_fitted_hp:
            # Use isotonic model for fitted range
            predictions[i] = iso_model.predict([[hp_val]])[0]
        else:
            # Use linear extrapolation beyond fitted range
            predictions[i] = slope * hp_val + intercept
        # Cap at asset limit
        predictions[i] = min(predictions[i], max_restriction_limit)
    
    return predictions

In [ ]:
# Package model with all parameters
model_data = {
    'iso': iso,
    'max_hp_fitted': fit_bin_centers.max(),
    'min_hp_fitted': fit_bin_centers.min(),
    'extrap_slope': extrapolation_slope,
    'extrap_intercept': extrapolation_intercept,
    'max_restriction_fitted': iso_pred.max(),
    'min_restriction_fitted': iso_pred.min(),
    'max_hp_asset': max_hp_asset,
    'max_restriction_asset': max_restriction_asset,
    'hp_at_max_restriction': hp_at_max_restriction
}

model_path = os.path.join(asset_dir, "model.pkl")
joblib.dump(model_data, model_path)
print(f"✓ Model saved to {model_path}")

# Save bin summary
bins_csv_path = os.path.join(asset_dir, f"{asset_id}_model_bins.csv")
with open(bins_csv_path, "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["hp_bin_center", "restriction_5th_percentile"])
    for center, val in zip(bin_centers, bin_restriction):
        writer.writerow([center, val])
print(f"✓ Bin data saved to {bins_csv_path}")

### 3.7 Visual Validation
Plot shows:
- Raw data (light blue scatter)
- 5th percentile bins (green dots)
- Isotonic fit (red solid)
- Linear extrapolation (red dashed)
- Asset limits (vertical/horizontal lines)

In [ ]:
plt.figure(figsize=(14, 8))

# Raw data
plt.scatter(hp, restriction, s=1, alpha=0.1, label="All Data", color='lightblue')

# Binned 5th percentile
plt.plot(fit_bin_centers, fit_bin_restriction, 'go', markersize=6, 
         label="5th Percentile per Bin", zorder=5)

# Isotonic fit
hp_plot = np.linspace(fit_bin_centers.min(), fit_bin_centers.max(), 500)
restriction_plot = iso.predict(hp_plot)
plt.plot(hp_plot, restriction_plot, 'r-', linewidth=2, 
         label="Isotonic Baseline (fitted)", zorder=4)

# Linear extrapolation
hp_extrap = np.linspace(fit_bin_centers.max(), max_hp_asset, 100)
restriction_extrap = extrapolation_slope * hp_extrap + extrapolation_intercept
restriction_extrap = np.minimum(restriction_extrap, max_restriction_asset)
plt.plot(hp_extrap, restriction_extrap, 'r--', linewidth=2, 
         label="Linear Extrapolation (forced)", zorder=4)

# Reference lines
plt.axvline(fit_bin_centers.max(), color='orange', linestyle=':', linewidth=1.5,
            label=f"Extrapolation starts at HP={fit_bin_centers.max():.0f}", zorder=3)
plt.axhline(max_restriction_asset, color='purple', linestyle='-.', linewidth=1.5, alpha=0.5,
            label=f"Asset Restriction Limit ({max_restriction_asset})", zorder=3)
plt.axvline(max_hp_asset, color='brown', linestyle='-.', linewidth=1.5, alpha=0.5,
            label=f"Asset HP Limit ({max_hp_asset})", zorder=3)

if hp_at_max_restriction <= max_hp_asset:
    plt.plot(hp_at_max_restriction, max_restriction_asset, 'rs', markersize=10,
             label=f"Reaches Max R at HP={hp_at_max_restriction:.0f}", zorder=6)

plt.xlabel("HydraulicHorsepower", fontsize=12)
plt.ylabel("AirFilterRestriction", fontsize=12)
plt.title(f"Clean Baseline Curve Fit: R_clean(HP) - Asset {asset_id}", fontsize=14)
plt.legend(loc='best', fontsize=9)
plt.grid(True, alpha=0.3)
plt.xlim(0, max_hp_asset * 1.05)
plt.ylim(0, max_restriction_asset * 1.1)
plt.tight_layout()

plot_path = os.path.join(asset_dir, f"{asset_id}_model_baseline.png")
plt.savefig(plot_path, dpi=150)
plt.show()
print(f"✓ Plot saved to {plot_path}")

### 3.8 Test Prediction Function
Verify model predictions across full HP range

In [ ]:
test_hps = [200, 500, 1000, 1200, 1500, 1800, 2000, int(max_hp_asset)]

print("="*60)
print("Testing R_clean(HP) predictions:")
print("="*60)

for hp_test in test_hps:
    r_pred = predict_restriction_from_hp(
        hp_test, iso, 
        fit_bin_centers.max(),
        extrapolation_slope, 
        extrapolation_intercept,
        max_restriction_asset
    )
    in_range = "✓ fitted" if hp_test <= fit_bin_centers.max() else "✗ extrapolated"
    print(f"HP={hp_test:4.0f} → R_clean={r_pred[0]:6.2f}  {in_range}")

## Step 4: Process All Assets
Wrap the above logic in a loop to process all assets

In [ ]:
# For production, wrap cells 6-14 in a loop:
for asset_id in asset_ids:
    print(f"\n{'='*60}\nProcessing asset {asset_id}\n{'='*60}")
    # ... (all the processing steps from cells 6-14)
    print(f"✓ Asset {asset_id} complete\n")

## Step 5: Create Combined Model File
Package all asset models into single file for deployment

In [ ]:
def combine_all_models(asset_ids, base_dir="api"):
    """Combine all asset models into single file"""
    models = {}
    for asset_id in asset_ids:
        model_path = os.path.join(base_dir, asset_id, "model.pkl")
        if os.path.exists(model_path):
            models[asset_id] = joblib.load(model_path)
        else:
            print(f"⚠️ Warning: {model_path} not found, skipping.")
    
    output_path = os.path.join(base_dir, "all_models.pkl")
    joblib.dump(models, output_path)
    print(f"✓ Combined model saved to {output_path}")
    print(f"✓ Total assets: {len(models)}")
    return models

all_models = combine_all_models(asset_ids)

## Summary

### What We Built
1. **Lower envelope extraction**: 5th percentile binning captures clean filter behavior
2. **Monotonic fitting**: Isotonic regression respects physical constraints
3. **Forced extrapolation**: Linear extension ensures full range coverage
4. **Asset-specific models**: Customized to each machine's operating envelope

### Key Assumptions
- ⚠️ Linear relationship holds in unobserved high-HP range
- ⚠️ 5th percentile adequately represents "clean" baseline
- ⚠️ Historical data contains sufficient clean filter examples

### Next Steps
- **Validation**: Compare predictions against known clean filter operation
- **Anomaly detection**: Use R_clean(HP) as baseline to flag degraded filters
- **Monitoring**: Track when actual restriction exceeds baseline by threshold
- **Refinement**: Update models as more clean baseline data becomes available

### Model Outputs
- `api/{asset}/model.pkl` - Serialized model + parameters
- `api/{asset}/{asset}_model_bins.csv` - Binned training data
- `api/{asset}/{asset}_model_baseline.png` - Visual validation
- `api/all_models.pkl` - Combined deployment artifact